In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
df = pd.read_csv("kw_email_use_selenium.csv")

url_list = df['agent_url'].unique()
def get_driver():
    options = webdriver.FirefoxOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Firefox(options=options)
    return driver


def collect_agent_contact_details(driver):
    try:
        agent_info = driver.find_element_by_class_name("AgentInformation")
    except:
        agent_info = ""
    try:
        info_container = agent_info.find_elements_by_class_name("AgentInformation__factBody")
    except:
        info_container = ""
    try:
        agent_email = [email.text for email in info_container if email.get_attribute("aria-label") == 'Agent E-mail'][0]
    except:
        agent_email = ""
    try:
        mobile_number = driver.find_element_by_class_name("AgentInformation__phoneMobileNumber").text
    except:
        mobile_number = ""
    try:
        office_number = driver.find_element_by_class_name("AgentInformation__phoneOfficeNumber").text
    except:
        office_number = ""
    try:
        agent_website = [i.text for i in info_container if i.get_attribute("aria-label") == 'Agent Website'][0]
    except:
        agent_website = ""
    contact_dict = {"agent_email":agent_email,"mobile_number":mobile_number,"office_number":office_number,'agent_website':agent_website }
    return contact_dict


def get_agent_details(driver):
    try:
        agent_licenses = driver.find_element_by_class_name("AgentContent__licenses").text
    except:
        agent_licenses = ""
    try:
        agent_bio = driver.find_element_by_class_name("AgentContent__bio").text
    except:
        agent_bio = ""
    try:
        serviceAreas = driver.find_element_by_class_name("AgentContent__serviceAreas").text
    except:
        serviceAreas = ""
    try:
        agent_team_name = driver.find_element_by_class_name("AgentContent__teamName").text
    except:
        agent_team_name = ""
    try:
        agent_team_info_ = driver.find_element_by_class_name("AgentContent__teamText").text.split("\n")
        agent_team_info = ", ".join(agent_team_info_)
    except:
        agent_team_info = ""
    try:
        logo_url = driver.find_element_by_class_name('AgentContent__teamAvatar').find_element_by_class_name("KWImage__image").get_attribute("src")
    except:
        logo_url = ""
    agent_details = {"agent_licenses":agent_licenses, "agent_bio":agent_bio, "serviceAreas":serviceAreas,"agent_team_name":agent_team_name,
                    "agent_team_info":agent_team_info,'logo_url':logo_url}
    return agent_details


def social_media_details(driver):
    try:
        social_media_container = driver.find_element_by_class_name("AgentInformation__socialMedia").find_elements_by_class_name("link")
    except:
        social_media_container = []
    social_media_links = [social_media.get_attribute("href") for social_media in social_media_container]
    try:
        facebook = [facebook for facebook in social_media_links if 'facebook' in facebook][0]
    except:
        facebook = ""
    try:
        instagram = [instagram for instagram in social_media_links if 'instagram' in instagram][0]
    except:
        instagram = ""
    try:
        twitter = [twitter for twitter in social_media_links if 'twitter' in twitter][0]
    except:
        twitter = ""
    try:
        linkedin = [linkedin for linkedin in social_media_links if 'linkedin' in linkedin][0]
    except:
        linkedin = ""
    social_media_dict = {"facebook":facebook,"instagram":instagram,"twitter":twitter,"linkedin":linkedin}
    social_media_links_str = "|".join(social_media_links)
    return social_media_dict,social_media_links_str


def collect_other_info(driver):
    container = driver.find_elements_by_class_name("AgentContent__section")
    try:
        Market_Cente = [i for i in container if 'Market Center' in i.text][0].find_element_by_class_name("AgentContent__sectionText").text
    except:
        Market_Cente = ""
    try:
        Languages = [i for i in container if 'Languages' in i.text][0].find_element_by_class_name("AgentContent__sectionText").text
    except:
        Languages = ""
    try:
        Specialties_designation = [i for i in container if 'Specialties and Designations' in i.text][0].find_element_by_class_name("AgentContent__sectionText").text
    except:
        Specialties_designation = ""
    other_info = {"Market_Cente":Market_Cente, "Languages":Languages, "Specialties_Designations":Specialties_designation}
    return other_info

def wait_to_page_load(driver):
    try:
        w = WebDriverWait(driver, 3)
        w.until(EC.presence_of_element_located((By.CLASS_NAME,"AgentContent__name")))
        print("Page load happened")
    except Exception as e:
        print("Timeout happened no page load")
    return driver

url_count = 47000
driver_count = 0
driver = get_driver()
for url in url_list[url_count:48000]:
    print(url)
    if driver_count >=100:
        print("need to close driver")
        driver.quit()
        driver_count = 0
        driver = get_driver()
        print("new driver initialize****************")
    driver.get(url)
    driver = wait_to_page_load(driver)
    try:
        agent_name = driver.find_element_by_class_name("AgentContent__name").text
    except:
        agent_name = ""
    try:
        time.sleep(0.5)
        profile_url = driver.find_element_by_class_name("AvatarImage__bg").get_attribute("style").replace('background-image: url("','').replace('");','')
    except:
        profile_url = ""
    try:
        tag = driver.find_element_by_class_name("pill").text
    except:
        tag = ""
    try:
        role = driver.find_element_by_class_name("AgentContent__team").text
    except:
        role = ""
    try:
        location = driver.find_element_by_class_name("AgentContent__location").text
    except:
        location = ""
    contact_dict = collect_agent_contact_details(driver)
    agent_details = get_agent_details(driver)
    social_media_dict,social_media_links_str = social_media_details(driver)
    other_info = collect_other_info(driver)
    data_dict = {'agent_url':url,"agent_name":agent_name, 'agent_role':role,'location':location,"agent_dp":profile_url, "tag":tag,"agent_email":contact_dict['agent_email'], "mobile_number":contact_dict['mobile_number'],"office_number":contact_dict['office_number'],'agent_license':agent_details['agent_licenses'], 'agent_bio':agent_details['agent_bio'],'serviceAreas':agent_details['serviceAreas'],'office_name':agent_details['agent_team_name'],'office_address':agent_details['agent_team_info'],'logo_url':agent_details['logo_url'],'facebook':social_media_dict['facebook'], 'instagram':social_media_dict['instagram'],'twitter':social_media_dict['twitter'],'linkedin':social_media_dict['linkedin'],'Market_Cente':other_info['Market_Cente'],'agent_language':other_info['Languages'],'Specialties_Designations':other_info['Specialties_Designations'],'social_media_links_str':social_media_links_str,'agent_website':contact_dict['agent_website']}
    data_df = pd.DataFrame(data_dict,index=[0],columns=['agent_url','agent_name', 'agent_role','location','agent_dp', 'tag','agent_email', 'mobile_number','office_number', 'agent_license', 'agent_bio', 'serviceAreas','office_name', 'office_address','logo_url','facebook', 'instagram', 'twitter', 'linkedin','Market_Cente', 'agent_language', 'Specialties_Designations','social_media_links_str','agent_website'])
    with open("kw_data-emailSelenium_47k_48k.csv",'a',newline='',encoding='utf-8') as f:
        data_df.to_csv(f, mode='a',header=f.tell()==0)
    print(url_count,url)
    url_count+=1
    driver_count+=1
    print("***********************************")
driver.quit()


https://kw.com/agent/UPA-6818563998966706176-5
Page load happened
47000 https://kw.com/agent/UPA-6818563998966706176-5
***********************************
https://kw.com/agent/UPA-6587385340484845571-1
Page load happened
47001 https://kw.com/agent/UPA-6587385340484845571-1
***********************************
https://kw.com/agent/UPA-6587385179106439169-1
Page load happened
47002 https://kw.com/agent/UPA-6587385179106439169-1
***********************************
https://kw.com/agent/UPA-6806603814323617792-9
Page load happened
47003 https://kw.com/agent/UPA-6806603814323617792-9
***********************************
https://kw.com/agent/UPA-6776164753526583296-5
Page load happened
47004 https://kw.com/agent/UPA-6776164753526583296-5
***********************************
https://kw.com/agent/UPA-6587384965238784002-2
Page load happened
47005 https://kw.com/agent/UPA-6587384965238784002-2
***********************************
https://kw.com/agent/UPA-6587385350929612806-9
Page load happened
4700

Page load happened
47053 https://kw.com/agent/UPA-6645717078395424770-1
***********************************
https://kw.com/agent/UPA-6587385452897751042-4
Timeout happened no page load
47054 https://kw.com/agent/UPA-6587385452897751042-4
***********************************
https://kw.com/agent/UPA-6779847898526380032-4
Page load happened
47055 https://kw.com/agent/UPA-6779847898526380032-4
***********************************
https://kw.com/agent/UPA-6587385286480510981-8
Page load happened
47056 https://kw.com/agent/UPA-6587385286480510981-8
***********************************
https://kw.com/agent/UPA-6810982817841217536-6
Page load happened
47057 https://kw.com/agent/UPA-6810982817841217536-6
***********************************
https://kw.com/agent/UPA-6587385055312605184-6
Page load happened
47058 https://kw.com/agent/UPA-6587385055312605184-6
***********************************
https://kw.com/agent/UPA-6620947830425235457-0
Page load happened
47059 https://kw.com/agent/UPA-662094783

Page load happened
47106 https://kw.com/agent/UPA-6691818428179492864-2
***********************************
https://kw.com/agent/UPA-6587384964277526533-9
Page load happened
47107 https://kw.com/agent/UPA-6587384964277526533-9
***********************************
https://kw.com/agent/UPA-6587385302302642176-1
Page load happened
47108 https://kw.com/agent/UPA-6587385302302642176-1
***********************************
https://kw.com/agent/UPA-6598891668176318470-3
Page load happened
47109 https://kw.com/agent/UPA-6598891668176318470-3
***********************************
https://kw.com/agent/UPA-6587385236075638785-1
Page load happened
47110 https://kw.com/agent/UPA-6587385236075638785-1
***********************************
https://kw.com/agent/UPA-6673653551340032000-3
Page load happened
47111 https://kw.com/agent/UPA-6673653551340032000-3
***********************************
https://kw.com/agent/UPA-6762031267100119040-1
Timeout happened no page load
47112 https://kw.com/agent/UPA-676203126

Page load happened
47159 https://kw.com/agent/UPA-6770743914168180736-2
***********************************
https://kw.com/agent/UPA-6587385238233542656-4
Page load happened
47160 https://kw.com/agent/UPA-6587385238233542656-4
***********************************
https://kw.com/agent/UPA-6587385182316498945-8
Page load happened
47161 https://kw.com/agent/UPA-6587385182316498945-8
***********************************
https://kw.com/agent/UPA-6734171554614398976-3
Page load happened
47162 https://kw.com/agent/UPA-6734171554614398976-3
***********************************
https://kw.com/agent/UPA-6587385191043895299-9
Page load happened
47163 https://kw.com/agent/UPA-6587385191043895299-9
***********************************
https://kw.com/agent/UPA-6587385098245296128-4
Page load happened
47164 https://kw.com/agent/UPA-6587385098245296128-4
***********************************
https://kw.com/agent/UPA-6807766466816393216-5
Page load happened
47165 https://kw.com/agent/UPA-6807766466816393216-

Page load happened
47212 https://kw.com/agent/UPA-6592244996652957698-3
***********************************
https://kw.com/agent/UPA-6587385101936168964-9
Page load happened
47213 https://kw.com/agent/UPA-6587385101936168964-9
***********************************
https://kw.com/agent/UPA-6742537737558941696-4
Page load happened
47214 https://kw.com/agent/UPA-6742537737558941696-4
***********************************
https://kw.com/agent/UPA-6702342959841665024-0
Page load happened
47215 https://kw.com/agent/UPA-6702342959841665024-0
***********************************
https://kw.com/agent/UPA-6788531143791185920-3
Page load happened
47216 https://kw.com/agent/UPA-6788531143791185920-3
***********************************
https://kw.com/agent/UPA-6587385261821243396-8
Page load happened
47217 https://kw.com/agent/UPA-6587385261821243396-8
***********************************
https://kw.com/agent/UPA-6587385235937226755-7
Page load happened
47218 https://kw.com/agent/UPA-6587385235937226755-

...

Page load happened
47742 https://kw.com/agent/UPA-6587385163988533250-6
***********************************
https://kw.com/agent/UPA-6675888572282560512-3
Page load happened
47743 https://kw.com/agent/UPA-6675888572282560512-3
***********************************
https://kw.com/agent/UPA-6587385314746765315-3
Page load happened
47744 https://kw.com/agent/UPA-6587385314746765315-3
***********************************
https://kw.com/agent/UPA-6587385221059440642-5
Page load happened
47745 https://kw.com/agent/UPA-6587385221059440642-5
***********************************
https://kw.com/agent/UPA-6707748764248002560-7
Page load happened
47746 https://kw.com/agent/UPA-6707748764248002560-7
***********************************
https://kw.com/agent/UPA-6675812272993890304-8
Page load happened
47747 https://kw.com/agent/UPA-6675812272993890304-8
***********************************
https://kw.com/agent/UPA-6587385404738166787-6
Page load happened
47748 https://kw.com/agent/UPA-6587385404738166787-

Page load happened
47795 https://kw.com/agent/UPA-6587385180162867203-2
***********************************
https://kw.com/agent/UPA-6657314681852715009-6
Page load happened
47796 https://kw.com/agent/UPA-6657314681852715009-6
***********************************
https://kw.com/agent/UPA-6587385372745601030-9
Page load happened
47797 https://kw.com/agent/UPA-6587385372745601030-9
***********************************
https://kw.com/agent/UPA-6587385227190190080-3
Page load happened
47798 https://kw.com/agent/UPA-6587385227190190080-3
***********************************
https://kw.com/agent/UPA-6744667623036141568-8
Page load happened
47799 https://kw.com/agent/UPA-6744667623036141568-8
***********************************
https://kw.com/agent/UPA-6587385255800016896-7
need to close driver
new driver initialize****************
Page load happened
47800 https://kw.com/agent/UPA-6587385255800016896-7
***********************************
https://kw.com/agent/UPA-6592936182038474755-4
Page load h

Page load happened
47848 https://kw.com/agent/UPA-6587385248929918979-2
***********************************
https://kw.com/agent/UPA-6587385385843826695-8
Page load happened
47849 https://kw.com/agent/UPA-6587385385843826695-8
***********************************
https://kw.com/agent/UPA-6587384967265255424-6
Page load happened
47850 https://kw.com/agent/UPA-6587384967265255424-6
***********************************
https://kw.com/agent/UPA-6643901809817714689-0
Page load happened
47851 https://kw.com/agent/UPA-6643901809817714689-0
***********************************
https://kw.com/agent/UPA-6587384956736446468-7
Page load happened
47852 https://kw.com/agent/UPA-6587384956736446468-7
***********************************
https://kw.com/agent/UPA-6587385086775300102-5
Page load happened
47853 https://kw.com/agent/UPA-6587385086775300102-5
***********************************
https://kw.com/agent/UPA-6587385393028255752-5
Page load happened
47854 https://kw.com/agent/UPA-6587385393028255752-

Page load happened
47901 https://kw.com/agent/UPA-6587385318695985155-3
***********************************
https://kw.com/agent/UPA-6587385243188121608-0
Page load happened
47902 https://kw.com/agent/UPA-6587385243188121608-0
***********************************
https://kw.com/agent/UPA-6825810258532106240-5
Page load happened
47903 https://kw.com/agent/UPA-6825810258532106240-5
***********************************
https://kw.com/agent/UPA-6839309419159699456-4
Page load happened
47904 https://kw.com/agent/UPA-6839309419159699456-4
***********************************
https://kw.com/agent/UPA-6592229873238167552-1
Page load happened
47905 https://kw.com/agent/UPA-6592229873238167552-1
***********************************
https://kw.com/agent/UPA-6847205147442487296-1
Page load happened
47906 https://kw.com/agent/UPA-6847205147442487296-1
***********************************
https://kw.com/agent/UPA-6633406914694144002-6
Page load happened
47907 https://kw.com/agent/UPA-6633406914694144002-

Page load happened
47954 https://kw.com/agent/UPA-6587385436002279429-1
***********************************
https://kw.com/agent/UPA-6587385407025446919-6
Page load happened
47955 https://kw.com/agent/UPA-6587385407025446919-6
***********************************
https://kw.com/agent/UPA-6587384995538583554-4
Page load happened
47956 https://kw.com/agent/UPA-6587384995538583554-4
***********************************
https://kw.com/agent/UPA-6833406357388075008-2
Page load happened
47957 https://kw.com/agent/UPA-6833406357388075008-2
***********************************
https://kw.com/agent/UPA-6721518306653970432-7
Page load happened
47958 https://kw.com/agent/UPA-6721518306653970432-7
***********************************
https://kw.com/agent/UPA-6587385309222604803-1
Page load happened
47959 https://kw.com/agent/UPA-6587385309222604803-1
***********************************
https://kw.com/agent/UPA-6587385362660478976-2
Page load happened
47960 https://kw.com/agent/UPA-6587385362660478976-